In [11]:
import numpy as np
import cv2
import os
import open3d as o3d
import matplotlib.pylab as plt

In [16]:
left_cap = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')
template_img = cv2.imread('./images/final_exam/Templates/Template-1.png')
while left_cap.isOpened():
    ret ,left_vdo = left_cap.read()
    if ret:
        left_vdo_gray = cv2.cvtColor(left_vdo, cv2.COLOR_BGR2GRAY)
        template_img_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)        
        sift = cv2.SIFT_create()
        bf = cv2.BFMatcher()   
        # def feature_object_detection(template_img, template_img_gray, left_vdo, left_vdo_gray, min_match_number) :
        # min_match_number = 10
        template_kpts, template_desc = sift.detectAndCompute(template_img_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(left_vdo_gray, None)# Find keypoints and descriptors directly
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])   
        if len(good_matches) >= 18 and len(good_matches) != 28  :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            # print(len(H))
            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.0) # H RANSAC
            print(len(good_matches))
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)
            # print (len(template_box))
            # print(len(transformed_box))              
            detected_img = cv2.polylines(left_vdo, [np.int32(transformed_box)], True, (255,0,50), 3, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
            # return detected_img, drawmatch_img
        #else :
            # print('Keypoints not enough')
            #return
        cv2.imshow('frame',left_vdo)
        # cv2.imshow('template_img_gray',template_img_gray)
        # cv2.imshow('frame_grat',left_vdo_gray)        
        if cv2.waitKey(33) & 0xff == 27 :
            break

left_cap.release()
cv2.destroyAllWindows()

24
20
24
24
19
19
20
20
20
21
21
21
22
20
20
20
20
19
23
18
19
25
24
19
25
23
20
24
24
21
22
19
21
24
24
24
19
20
21
25
23
20
18
19
19
19
19
18
18
20
19
21
18
19
20
20
18
24
22
19
18
21
22
20
23
22
23
18
20
18
23
25
18
18
19
22
23
20
20
18
19
22
20
23
19
20
20
19
22
26
23
18
24
18
23
18
19
23
19
19
18
22
31
20
23
18
23
24
23
19
22
23
20
20
20
22
18
24
19
20
26
21
23
19
19
19
19
20
18
23
20
20
18
19
23
24
20
20
20
19
20
18
18
19
23
22
21
26
22
18
18
19
25
29
33
30
33
36
34
27
32
41
30
26
35
33
26
30
36
41
31
33
44
40
41
50
50
34
48
50
57
49
43
54
43
45
44
51
40
50
45
42
35
52
53
52
57
67
54
63
59
62
69
75
62
60
58
54
71
57
69
67
77
73
57
75
60
73
63
69
69
72
66
75
66
63
62
57
61
67
64
55
48
55
50
62
51
47
46
50
48
45
59
58
57
46
43
48
40
33
51
49
32
31
55
51
25
44
36
31
26
21
21
18
21
24
26
36
25
33
33
34
26
30
30
34
32
36
27
21
21
24
23
27
31
24
21
20
23
19
18
32
36
52
53
39
39
26
29
26
19
18
18
21
21
26
26
23
26
22
24
19
21
21
18
21
18
19
18
32
46
48
55
57
48
55
65
48
49
53
58
55
53
5